In [1]:
!export LC_ALL=en_US.UTF-8
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')
np.random.seed(0)

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to /home/ra2630/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ra2630/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
X_train_comp_all = []
X_train_comp_with_answer_marked_all = []
X_train_ans_all = []
X_train_comp_answer_label_all = []
Y_train_ques_all = []
invalid = 0
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            invalid=invalid+1
            continue
        a_lab[start:end] = 1
        
              
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            start,end = find_sub_list(answer,passage)
            if start == -1:
                invalid = invalid+1
                continue
            marked_comp = np.zeros(len(passage))
            marked_comp[start:end] = 1
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_all.append(cappedPassage)
            X_train_comp_with_answer_marked_all.append(marked_comp)
            X_train_ans_all.append(answer)
            X_train_comp_answer_label_all.append(a_lab)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1

In [8]:
c = list(zip(X_train_comp_all,X_train_ans_all, Y_train_ques_all,X_train_comp_with_answer_marked_all, X_train_comp_answer_label_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_ans_all_shuffled, Y_train_ques_all_shuffled, X_train_comp_with_answer_marked_all_shuffled, X_train_comp_answer_label_shuffled = zip(*c)


In [9]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all_shuffled[0:400] for item in sublist] + [item for sublist in Y_train_ques_all_shuffled[0:400] for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [10]:
wordToTake = 4000
words = findKMostFrequentWords(100000)

In [11]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(4000, 300)

In [12]:
!hostname
print(invalid)

cuda2.cims.nyu.edu
4858


In [13]:
examples_to_take_train = 320

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_all_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
X_train_comp_with_answer_marked = X_train_comp_with_answer_marked_all_shuffled[0:examples_to_take_train]
X_train_comp_answer_label = X_train_comp_answer_label_shuffled[0:examples_to_take_train]

In [14]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [15]:
X_train_comp_with_answer_marked[0], max_document_len

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [16]:
document_tokens = np.full((examples_to_take_train, max_document_len), END_TOKEN,dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

answer_labels_all = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)


In [17]:
for i in range(examples_to_take_train):
    answer_labels_all[i,0:len(X_train_comp_answer_label[i])] = X_train_comp_answer_label[i]
    answer_labels[i, 0:len(X_train_comp_with_answer_marked[i])] = X_train_comp_with_answer_marked[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    answer_lengths[i] = len(X_train_ans[i])
    
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)    

In [18]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [19]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(4000, 300)
8700
565
8506


In [20]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [21]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [22]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [23]:
max_document_len

344

## Batch Data Preparation

In [24]:
max_question_len

29

In [25]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels_all':[],
                    'answer_labels':[],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[]}
        
        for index,inp in enumerate(inputs):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max_question_len
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels_all'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==4:
                output['answer_lengths'].append(inp[start:end])
            elif index==5:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==7:
                output['question_lengths'].append(inp[start:end])
        
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels_all"] = np.array(output["answer_labels_all"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_lengths"] = np.array(output["question_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        outputs.append(output)
        start = start + batch_size
            
    return outputs


In [26]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels_all,answer_labels,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)
for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze()
number_of_batches = len(batch_input)
print("Number of batches = ", number_of_batches)

Number of batches =  10


In [27]:
batch_input_test = batch_input[7:]
batch_input = batch_input[0:7]

## Model

In [28]:
len(batch_input)

7

In [29]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import time

In [30]:
use_cuda = torch.cuda.is_available()
use_attention = False

In [31]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [32]:
class AnswerEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(AnswerEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True, bidirectional=True) #Input_size = Hidden_Size
        if use_cuda:
            self.gru = self.gru.cuda()
        self.fc = nn.Linear(hidden_size*2, 1)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        final_output = self.fc(output)
        final_output = F.sigmoid(final_output)
        self.hiddenState = hidden
        return final_output, output, hidden
    
    def initHidden(self, batch_size):
        result = Variable(torch.zeros(2, batch_size, self.hidden_size)) #2 for BiDirectional
        if use_cuda:
            result = result.cuda()
        return result

In [33]:
class QuestionEncoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
    
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden

In [34]:
class QuestionDecoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
        
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    
class FCLayer(nn.Module):
    def __init__(self,input_size, output_size):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()
        
    def forward(self, x):
        return self.fc(x)

class QuestionGenerationFC(nn.Module):
    def __init__(self,input_size, output_size):
        super(QuestionGenerationFC, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x):
        output = self.fc(x)
        output = F.log_softmax(output, dim=-1)
        return output

In [35]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, max_document_len, dropout_p=0.1):
        
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_document_len = max_document_len

        self.attn_reduce = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.attn_combine = nn.Linear(self.max_document_len + self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        
        if use_cuda:
            self.attn_reduce = self.attn_reduce.cuda()
            self.attn_combine = self.attn_combine.cuda()
            self.dropout = self.dropout.cuda()
        

    def forward(self, input, hidden, encoder_outputs):

        hidden = hidden.squeeze(0)
        hidden = self.dropout(hidden)
        concat = torch.cat((input.unsqueeze(0), hidden.unsqueeze(0)), 1)
        concat_reduced = self.attn_reduce(concat)
        attn_weights = F.softmax(concat_reduced, dim=1)
        attn_applied = torch.mm(encoder_outputs, attn_weights.squeeze(0).unsqueeze(1))
        output = torch.cat((hidden.unsqueeze(0), attn_applied.squeeze(1).unsqueeze(0)), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        self.hidden_state = hidden
        return output, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [36]:
hidden_size = reduced_glove.shape[1]

embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
fcLayer = FCLayer(max_document_len+1, 1)
answerEncoder = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder1 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionEncoder2 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)

questionDecoder = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
attention = AttnDecoderRNN(hidden_size= hidden_size,max_document_len = max_document_len)

answerEncoder.train()
questionEncoder1.train()
questionEncoder2.train()
questionDecoder.train()
questionGenerator.train()
attention.train()

train_param = []

for model in [answerEncoder, questionEncoder1, questionEncoder2, questionDecoder, questionGenerator, attention, fcLayer]:
    train_param += [p for p in model.parameters() if p.requires_grad]

print("Number of trainable parameters = ", len(train_param))

optimizer = torch.optim.Adam(train_param, 0.01)
criterion1 = nn.BCELoss()
#criterion2 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
#criterion2 = nn.MultiLabelSoftMarginLoss()

Number of trainable parameters =  30


In [37]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [ ]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
question_encoder_1_hidden = None
question_encoder_2_hidden = None
question_decoder_hidden = None
attn_output = None
attention_hidden = None
Attention_Weights = None

for epoch in range(1, num_epochs+1):
    avg_loss = 0
    epoch_time_start = time.time()
    for batch_num in range(len(batch_input)):
        batch_time_start = time.time()
        answer_encoder_hidden = answerEncoder.initHidden(batch_size)
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        if type(question_encoder_1_hidden) == Variable:
            question_encoder_1_hidden = repackage_hidden(question_encoder_1_hidden)
        if type(question_encoder_2_hidden) == Variable:
            question_encoder_2_hidden = repackage_hidden(question_encoder_2_hidden)
        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        if(type(attn_output) == Variable):
            attn_output = repackage_hidden(attn_output)
        if(type(attention_hidden) == Variable):
            attention_hidden = repackage_hidden(attention_hidden)
        if(type(Attention_Weights) == Variable):
            Attention_Weights = repackage_hidden(Attention_Weights)
        
        current_batch_size = len(batch_input[batch_num]["document_tokens"])
        if current_batch_size != batch_size:
            continue
        
        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        
        for i in range(current_batch_size):
            mask[i,0:batch_input[batch_num]["document_lengths"][i]] = 1
            

        inp = Variable(torch.from_numpy(batch_input[batch_num]["document_tokens"]).long())

        labels = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels_all"])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        
        #answer_outputs = Variable(torch.zeros(current_batch_size, max_document_len, hidden_size))
        #answer_tags = Variable(torch.zeros(current_batch_size, max_document_len, 1))
        
        if use_cuda:
            #answer_outputs = answer_outputs.cuda()
            #answer_tags = answer_tags.cuda()
            embedded_inp = embedded_inp.cuda()

        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)              

        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


        answer_loss = criterion1(outputs, labels.float())
        
        
        
        
        
        answer_mask_1 = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        answer_mask_2 = Variable(torch.from_numpy(1 - batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        
        if use_cuda:
            answer_mask_1 = answer_mask_1.cuda()
            answer_mask_2 = answer_mask_2.cuda()
        
        question_encoder_input1 = torch.mul(answer_mask_1, answer_outputs.float())
        question_encoder_input2 = torch.mul(answer_mask_2, answer_outputs.float())
        
        question_encoder_1_hidden = answer_encoder_hidden.view(1,answer_encoder_hidden.shape[1],2*answer_encoder_hidden.shape[2])
        question_encoder_2_hidden = answer_encoder_hidden.view(1,answer_encoder_hidden.shape[1],2*answer_encoder_hidden.shape[2])
        
        question_encoder_1_outputs , question_encoder_1_hidden = questionEncoder1(question_encoder_input1, question_encoder_1_hidden)
        question_encoder_2_outputs , question_encoder_2_hidden = questionEncoder2(question_encoder_input2, question_encoder_2_hidden)
            
        
        

        question_loss = 0
        question_decoder_hidden = question_encoder_2_hidden
        embedded_inputs = embedder(torch.from_numpy(batch_input[batch_num]["question_input_tokens"]).long())
        output_labels = Variable(torch.from_numpy(batch_input[batch_num]["question_output_tokens"]).long())
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()

        for quesL in range(batch_input[batch_num]["question_input_tokens"].shape[1]):
            if use_attention:
                attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
                decoder_output, attention_hidden = questionDecoder(
                    embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
            else:
                '''
                decoder_output, question_decoder_hidden = questionDecoder(
                    embedded_inputs[:,quesL:quesL+1,:],
                    question_decoder_hidden)
                '''
                decoder_output, question_decoder_hidden = questionDecoder(
                    torch.cat((embedded_inputs[:,quesL:quesL+1,:], embedded_inp), dim=1),
                    question_decoder_hidden)

                decoder_output = fcLayer(decoder_output.resize(batch_size,300,max_document_len+1)).resize(batch_size, 1, 300)

            final_output = questionGenerator(decoder_output.squeeze(1))
            
            '''
            idx = np.where (output_labels[:, quesL:quesL+1].squeeze(1).data == END_TOKEN)[0].tolist()
            final_copy = final_output.clone()
            if len(idx):
                print(final_copy.data[idx])
                final_copy.data[idx] = 0
                final_copy.data[idx][:,END_TOKEN] = 1
                print(final_copy[idx])
                print("IDX", idx)
                print("Where:", np.where(final_copy[idx].data == 1)) 
            '''
            question_loss += criterion2(final_output,
                                       output_labels[:, quesL:quesL+1].squeeze(1))
            


        net_loss = answer_loss + question_loss
        net_loss.backward()
        optimizer.step()


        avg_loss+= net_loss.data[0]
        batch_time_end = time.time()
        
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f \t Time Taken: %d seconds'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0], batch_time_end-batch_time_start))
    epoch_time_end = time.time()



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f Time Taken: %d seconds'
                   %(epoch, avg_loss/len(batch_input), (epoch_time_end-epoch_time_start)))
        

Batch: 0 	 Epoch : 1	Net Loss: 239.0333 	Answer Loss: 0.2605 	Question Loss: 238.7728 	 Time Taken: 4 seconds
Batch: 1 	 Epoch : 1	Net Loss: 143.0057 	Answer Loss: 0.1259 	Question Loss: 142.8798 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 1	Net Loss: 141.9254 	Answer Loss: 0.1393 	Question Loss: 141.7861 	 Time Taken: 4 seconds
Batch: 3 	 Epoch : 1	Net Loss: 149.3813 	Answer Loss: 0.1663 	Question Loss: 149.2150 	 Time Taken: 4 seconds
Batch: 4 	 Epoch : 1	Net Loss: 115.0185 	Answer Loss: 0.1973 	Question Loss: 114.8212 	 Time Taken: 4 seconds
Batch: 5 	 Epoch : 1	Net Loss: 112.2680 	Answer Loss: 0.1371 	Question Loss: 112.1308 	 Time Taken: 4 seconds
Batch: 6 	 Epoch : 1	Net Loss: 95.5194 	Answer Loss: 0.1225 	Question Loss: 95.3970 	 Time Taken: 4 seconds
Average Loss after Epoch 1 : 142.3074 Time Taken: 30 seconds
Batch: 0 	 Epoch : 2	Net Loss: 101.5938 	Answer Loss: 0.1219 	Question Loss: 101.4718 	 Time Taken: 4 seconds
Batch: 1 	 Epoch : 2	Net Loss: 100.0057 	Answer Loss: 0.1157 

Batch: 1 	 Epoch : 11	Net Loss: 43.2659 	Answer Loss: 0.1043 	Question Loss: 43.1616 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 11	Net Loss: 43.1205 	Answer Loss: 0.1147 	Question Loss: 43.0058 	 Time Taken: 4 seconds
Batch: 3 	 Epoch : 11	Net Loss: 40.4991 	Answer Loss: 0.1170 	Question Loss: 40.3821 	 Time Taken: 4 seconds
Batch: 4 	 Epoch : 11	Net Loss: 39.3364 	Answer Loss: 0.1460 	Question Loss: 39.1904 	 Time Taken: 4 seconds
Batch: 5 	 Epoch : 11	Net Loss: 44.0989 	Answer Loss: 0.1174 	Question Loss: 43.9814 	 Time Taken: 4 seconds
Batch: 6 	 Epoch : 11	Net Loss: 40.2468 	Answer Loss: 0.1111 	Question Loss: 40.1357 	 Time Taken: 4 seconds
Average Loss after Epoch 11 : 42.3743 Time Taken: 30 seconds
Batch: 0 	 Epoch : 12	Net Loss: 47.2956 	Answer Loss: 0.1075 	Question Loss: 47.1881 	 Time Taken: 4 seconds
Batch: 1 	 Epoch : 12	Net Loss: 40.2735 	Answer Loss: 0.1004 	Question Loss: 40.1731 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 12	Net Loss: 36.0073 	Answer Loss: 0.1086 	Quest

Batch: 1 	 Epoch : 21	Net Loss: 31.8616 	Answer Loss: 0.0813 	Question Loss: 31.7803 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 21	Net Loss: 28.9569 	Answer Loss: 0.0843 	Question Loss: 28.8726 	 Time Taken: 4 seconds
Batch: 3 	 Epoch : 21	Net Loss: 26.5270 	Answer Loss: 0.0875 	Question Loss: 26.4396 	 Time Taken: 4 seconds
Batch: 4 	 Epoch : 21	Net Loss: 26.0640 	Answer Loss: 0.1214 	Question Loss: 25.9427 	 Time Taken: 4 seconds
Batch: 5 	 Epoch : 21	Net Loss: 31.6695 	Answer Loss: 0.0755 	Question Loss: 31.5941 	 Time Taken: 4 seconds
Batch: 6 	 Epoch : 21	Net Loss: 26.2904 	Answer Loss: 0.0786 	Question Loss: 26.2118 	 Time Taken: 4 seconds
Average Loss after Epoch 21 : 29.3452 Time Taken: 30 seconds
Batch: 0 	 Epoch : 22	Net Loss: 27.3374 	Answer Loss: 0.0871 	Question Loss: 27.2503 	 Time Taken: 4 seconds
Batch: 1 	 Epoch : 22	Net Loss: 26.2150 	Answer Loss: 0.0829 	Question Loss: 26.1322 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 22	Net Loss: 24.2686 	Answer Loss: 0.0740 	Quest

Batch: 1 	 Epoch : 31	Net Loss: 21.7127 	Answer Loss: 0.0540 	Question Loss: 21.6588 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 31	Net Loss: 19.7194 	Answer Loss: 0.0479 	Question Loss: 19.6715 	 Time Taken: 4 seconds
Batch: 3 	 Epoch : 31	Net Loss: 16.6439 	Answer Loss: 0.0512 	Question Loss: 16.5926 	 Time Taken: 4 seconds
Batch: 4 	 Epoch : 31	Net Loss: 13.7047 	Answer Loss: 0.0560 	Question Loss: 13.6487 	 Time Taken: 4 seconds
Batch: 5 	 Epoch : 31	Net Loss: 17.0293 	Answer Loss: 0.0501 	Question Loss: 16.9792 	 Time Taken: 4 seconds
Batch: 6 	 Epoch : 31	Net Loss: 12.6424 	Answer Loss: 0.0433 	Question Loss: 12.5991 	 Time Taken: 4 seconds
Average Loss after Epoch 31 : 16.9021 Time Taken: 30 seconds
Batch: 0 	 Epoch : 32	Net Loss: 16.3518 	Answer Loss: 0.0530 	Question Loss: 16.2987 	 Time Taken: 4 seconds
Batch: 1 	 Epoch : 32	Net Loss: 19.0603 	Answer Loss: 0.0540 	Question Loss: 19.0063 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 32	Net Loss: 15.4563 	Answer Loss: 0.0421 	Quest

Batch: 2 	 Epoch : 41	Net Loss: 6.9994 	Answer Loss: 0.0239 	Question Loss: 6.9756 	 Time Taken: 4 seconds
Batch: 3 	 Epoch : 41	Net Loss: 4.7456 	Answer Loss: 0.0241 	Question Loss: 4.7214 	 Time Taken: 4 seconds
Batch: 4 	 Epoch : 41	Net Loss: 4.2684 	Answer Loss: 0.0348 	Question Loss: 4.2336 	 Time Taken: 4 seconds
Batch: 5 	 Epoch : 41	Net Loss: 5.2605 	Answer Loss: 0.0331 	Question Loss: 5.2273 	 Time Taken: 4 seconds
Batch: 6 	 Epoch : 41	Net Loss: 3.7979 	Answer Loss: 0.0219 	Question Loss: 3.7760 	 Time Taken: 4 seconds
Average Loss after Epoch 41 : 5.5968 Time Taken: 30 seconds
Batch: 0 	 Epoch : 42	Net Loss: 4.0210 	Answer Loss: 0.0239 	Question Loss: 3.9971 	 Time Taken: 4 seconds
Batch: 1 	 Epoch : 42	Net Loss: 4.7974 	Answer Loss: 0.0255 	Question Loss: 4.7719 	 Time Taken: 4 seconds
Batch: 2 	 Epoch : 42	Net Loss: 3.5843 	Answer Loss: 0.0260 	Question Loss: 3.5582 	 Time Taken: 4 seconds
Batch: 3 	 Epoch : 42	Net Loss: 3.0743 	Answer Loss: 0.0255 	Question Loss: 3.0487 	

In [ ]:
fcLayer(x.cpu().view(32,300,2)).shape

In [ ]:
y.shape

In [ ]:
.shape

In [ ]:
checkpoint = torch.load("/data/ra2630/qgen_base_all_data.pt")

In [ ]:
reduced_glove.shape[1]

In [ ]:
embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
answerEncoder2 = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder2_1 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionEncoder2_2 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder2 = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator2 = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
attention2 = AttnDecoderRNN(hidden_size= hidden_size,max_document_len = max_document_len)
fcLayer2 = FCLayer(max_document_len+1, 1)
optimizer2 = torch.optim.Adam(train_param, 0.01)


#answerEncoder2.load_state_dict(checkpoint["answerEncoder"])
#questionEncoder2_1.load_state_dict(checkpoint["questionEncoder1"])
#questionEncoder2_2.load_state_dict(checkpoint["questionEncoder2"])
#questionDecoder2.load_state_dict(checkpoint["questionDecoder"])
#questionGenerator2.load_state_dict(checkpoint["questionGenerator"])
#optimizer2.load_state_dict(checkpoint["optimizer"])


answerEncoder2 = answerEncoder
questionEncoder2_1 = questionEncoder1
questionEncoder2_2 = questionEncoder2
questionDecoder2 = questionDecoder
questionGenerator2 = questionGenerator
attention2 = attention
fcLayer2 = fcLayer
optimizer2 = optimizer



In [ ]:
def prepareSetForInferenceFromBatch(batch_number):
    data_dict = batch_input[batch_number]
    data_dict['answer_mask_1'] = data_dict['answer_labels']
    data_dict['answer_mask_2'] = 1 - data_dict['answer_mask_1']
    return data_dict

In [ ]:
def prepareSetForInference(context, answer, maxDocLength, batch_size):
    
    passage = word_tokenize(context.lower())
    doc_len = len(passage)
    
    ans_condition = word_tokenize(answer.lower())
    
    ans_len = len(ans_condition)
        
    start,end = find_sub_list(ans_condition,passage)

    if start == -1:
            print("Couldn't Find answer in text, Please try again !!")
            return None
    
    
    document_token = np.full((1, maxDocLength), END_TOKEN, dtype=np.int32)
    document_length = np.zeros(1, dtype=np.int32)
    answer_mask_1 = np.zeros((1,maxDocLength), dtype=np.int32)
    answer_mask_2 = np.zeros((1,maxDocLength), dtype=np.int32)
    answer_length = np.zeros(1, dtype=np.int32)
    
    document_length[0] = doc_len
    answer_length[0] = ans_len
    answer_mask_1[0,start:end] = 1
    answer_mask_2 = 1 - answer_mask_1
    
    
    for i in range(doc_len):
        document_token[:,i] = look_up_word_reduced(passage[i])
    
    document_token = document_token.repeat(batch_size, axis = 0)
    document_length = document_length.repeat(batch_size, axis = 0)
    answer_mask_1 = answer_mask_1.repeat(batch_size, axis = 0)
    answer_mask_2 = answer_mask_2.repeat(batch_size, axis = 0)
    answer_length = answer_length.repeat(batch_size, axis = 0)
    
    
    return {
        "document_tokens" : document_token,
        "document_length" : document_length,
        "answer_mask_1" : answer_mask_1,
        "answer_mask_2" : answer_mask_2,
        "answer_length" : answer_length,
    }

In [ ]:
def inference(context, answer, maxDocLen, batch_num = 0):

    #data_dict = prepareSetForInference(context, answer, maxDocLen, batch_size)
    data_dict = prepareSetForInferenceFromBatch(batch_num)
    use_attention = False
    use_cuda = True

    answer_encoder_hidden_inf = answerEncoder2.initHidden(batch_size)


    inp = Variable(torch.from_numpy(data_dict["document_tokens"]).long())
    labels = Variable(torch.from_numpy(data_dict["answer_mask_1"])).long() #Let's see if we want to use or not
    embedded_inp = embedder(inp)


    if use_cuda:
        labels = labels.cuda()
        embedded_inp = embedded_inp.cuda()


    answer_encoder_hidden_inf = repackage_hidden(answer_encoder_hidden_inf)
    answer_tags, answer_outputs, answer_encoder_hidden_inf = answerEncoder(embedded_inp, answer_encoder_hidden_inf)

    if use_cuda:
        answer_outputs = answer_outputs.cuda()
        answer_tags = answer_tags.cuda()

    ####USE ANSWER_TAGS TO CONDITION QUESTIONS RHATHRE THAN PASSING ANSWERS

    answer_mask_1 = Variable(torch.from_numpy(data_dict["answer_mask_1"])).float().unsqueeze(-1)
    answer_mask_2 = Variable(torch.from_numpy(data_dict["answer_mask_2"])).float().unsqueeze(-1)

    if use_cuda:
        answer_mask_1 = answer_mask_1.cuda()
        answer_mask_2 = answer_mask_2.cuda()

    question_encoder_input1 = torch.mul(answer_mask_1, answer_outputs.float())
    question_encoder_input2 = torch.mul(answer_mask_2, answer_outputs.float())

    if use_cuda:
        question_encoder_input1 = question_encoder_input1.cuda()
        question_encoder_input2 = question_encoder_input2.cuda()

    question_encoder_hidden_1_inf = answer_encoder_hidden_inf.view(1,answer_encoder_hidden_inf.shape[1],2*answer_encoder_hidden_inf.shape[2])
    question_encoder_hidden_2_inf = answer_encoder_hidden_inf.view(1,answer_encoder_hidden_inf.shape[1],2*answer_encoder_hidden_inf.shape[2])

    question_encoder_1_outputs , question_encoder_1_hidden = questionEncoder1(question_encoder_input1, question_encoder_hidden_1_inf)
    question_encoder_2_outputs , question_encoder_2_hidden = questionEncoder2(question_encoder_input2, question_encoder_hidden_2_inf)



    maxGenQuesLen = 20
    currGenQuestionLen = 0

    current_question_token = torch.from_numpy(np.full((batch_size), START_TOKEN)).long()
    questionGenerated = []

    for i in range(batch_size):
        questionGenerated.append([])

    question_decoder_hidden_inf = question_encoder_2_hidden
    while currGenQuestionLen < maxGenQuesLen:
        question_token_embedding = embedder(current_question_token)
        if use_cuda:
            question_token_embedding = question_token_embedding.cuda()

        if use_attention:
            attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
            decoder_output, attention_hidden = questionDecoder(
                embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
        else:
            decoder_output, question_decoder_hidden_inf = questionDecoder(
                    torch.cat((question_token_embedding.unsqueeze(1), embedded_inp), dim=1),
                    question_decoder_hidden_inf)

            decoder_output = fcLayer(decoder_output.resize(batch_size,300,max_document_len+1)).resize(batch_size, 1, 300)


        final_output = questionGenerator2(decoder_output.squeeze(1))
        current_question_token = np.argmax(final_output.data, axis = 1)
        for i in range(batch_size):
            questionGenerated[i].append(look_up_token_reduced(current_question_token[i]))
        currGenQuestionLen += 1

    return questionGenerated

In [ ]:
b_n = 8

questionGenerated = inference(' '.join(X_train_comp_all_shuffled[10]),
                              ' '.join(X_train_comp_all_shuffled[10]),
                              max_document_len, b_n)

for i in range(32):

    #print(' '.join(X_train_comp_all_shuffled[10]))
    #print(' '.join(X_train_comp_all_shuffled[10]))
    print(' '.join(Y_train_ques_all_shuffled[32*b_n + i]))
    
    for questions in questionGenerated[i:i+1]:
        print("Generated Question : ")
        for word in questions:
            print(word, sep = ' ', end = ' ')
        print("")
         


In [ ]:
for questions in questionGenerated[0:1]:
    print("Question : ")
    for word in questions:
        print(word, sep = ' ', end = ' ')
    print("")
    
for questions in questionGenerated[-2:-1]:
    print("Question : ")
    for word in questions:
        print(word, sep = ' ', end = ' ')
    print("")

In [ ]:
np.argmax(final_output.data, axis = 1).shape

In [ ]:
(data_dict["document_tokens"])

In [ ]:
for i in np.argmax(final_output.data, axis = 1):
    print(look_up_token_reduced(i))

In [ ]:
np.full((batch_size), START_TOKEN).shape

In [ ]:
batch_num = 3
example_num = 0
printQuestion(batch_num, example_num)
printAnswer(batch_num, example_num)
printContext(batch_num, example_num)

In [ ]:
' '.join(X_train_ans_all_shuffled[0 + 19])

In [ ]:
' '.join(Y_train_ques_all_shuffled[0 + 20])

In [ ]:
' '.join(X_train_comp_all_shuffled[0 + 20])

In [ ]:
batch_input[0]['question_input_tokens'][0], batch_input[0]['question_output_tokens'][0]

In [ ]:
for b in output_labels:
    for i in b:
        print(look_up_token_reduced(i.cpu().data[0]), sep = " ", end = " ")
    print(" ")

In [ ]:
avg_loss = 0
    epoch_time_start = time.time()
    for batch_num in range(len(batch_input)):
        batch_time_start = time.time()
        answer_encoder_hidden = answerEncoder.initHidden(batch_size)
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        if type(question_encoder_1_hidden) == Variable:
            question_encoder_1_hidden = repackage_hidden(question_encoder_1_hidden)
        if type(question_encoder_2_hidden) == Variable:
            question_encoder_2_hidden = repackage_hidden(question_encoder_2_hidden)
        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        if(type(attn_output) == Variable):
            attn_output = repackage_hidden(attn_output)
        if(type(attention_hidden) == Variable):
            attention_hidden = repackage_hidden(attention_hidden)
        if(type(Attention_Weights) == Variable):
            Attention_Weights = repackage_hidden(Attention_Weights)
        
        current_batch_size = len(batch_input[batch_num]["document_tokens"])
        if current_batch_size != batch_size:
            continue
        
        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        
        for i in range(current_batch_size):
            mask[i,0:batch_input[batch_num]["document_lengths"][i]] = 1
            

        inp = Variable(torch.from_numpy(batch_input[batch_num]["document_tokens"]).long())

        labels = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels_all"])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        
        #answer_outputs = Variable(torch.zeros(current_batch_size, max_document_len, hidden_size))
        #answer_tags = Variable(torch.zeros(current_batch_size, max_document_len, 1))
        
        if use_cuda:
            #answer_outputs = answer_outputs.cuda()
            #answer_tags = answer_tags.cuda()
            embedded_inp = embedded_inp.cuda()

        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)              

        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


        answer_loss = criterion1(outputs, labels.float())
        
        
        
        
        
        answer_mask_1 = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        answer_mask_2 = Variable(torch.from_numpy(1 - batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        
        if use_cuda:
            answer_mask_1 = answer_mask_1.cuda()
            answer_mask_2 = answer_mask_2.cuda()
        
        question_encoder_input1 = torch.mul(answer_mask_1, answer_outputs.float())
        question_encoder_input2 = torch.mul(answer_mask_2, answer_outputs.float())
        
        question_encoder_1_hidden = answer_encoder_hidden.view(1,answer_encoder_hidden.shape[1],2*answer_encoder_hidden.shape[2])
        question_encoder_2_hidden = answer_encoder_hidden.view(1,answer_encoder_hidden.shape[1],2*answer_encoder_hidden.shape[2])
        
        question_encoder_1_outputs , question_encoder_1_hidden = questionEncoder1(question_encoder_input1, question_encoder_1_hidden)
        question_encoder_2_outputs , question_encoder_2_hidden = questionEncoder2(question_encoder_input2, question_encoder_2_hidden)
            
        
        

        question_loss = 0
        question_decoder_hidden = question_encoder_2_hidden
        embedded_inputs = embedder(torch.from_numpy(batch_input[batch_num]["question_input_tokens"]).long())
        output_labels = Variable(torch.from_numpy(batch_input[batch_num]["question_output_tokens"]).long())
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()

        for quesL in range(batch_input[batch_num]["question_input_tokens"].shape[1]):
            if use_attention:
                attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
                decoder_output, attention_hidden = questionDecoder(
                    embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
            else:
                '''
                decoder_output, question_decoder_hidden = questionDecoder(
                    embedded_inputs[:,quesL:quesL+1,:],
                    question_decoder_hidden)
                '''
                decoder_output, question_decoder_hidden = questionDecoder(
                    torch.cat((embedded_inputs[:,quesL:quesL+1,:], embedded_inp), dim=1),
                    question_decoder_hidden)

                decoder_output = fcLayer(x.resize(batch_size,300,max_document_len+1)).resize(batch_size, 1, 300)

            final_output = questionGenerator(decoder_output.squeeze(1))
            
            '''
            idx = np.where (output_labels[:, quesL:quesL+1].squeeze(1).data == END_TOKEN)[0].tolist()
            final_copy = final_output.clone()
            if len(idx):
                print(final_copy.data[idx])
                final_copy.data[idx] = 0
                final_copy.data[idx][:,END_TOKEN] = 1
                print(final_copy[idx])
                print("IDX", idx)
                print("Where:", np.where(final_copy[idx].data == 1)) 
            '''
            question_loss += criterion2(final_output,
                                       output_labels[:, quesL:quesL+1].squeeze(1))
            


        net_loss = answer_loss + question_loss
        net_loss.backward()
        optimizer.step()


        avg_loss+= net_loss.data[0]
        batch_time_end = time.time()
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f \t Time Taken: %d seconds'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0], batch_time_end-batch_time_start))
    epoch_time_end = time.time()



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f Time Taken: %d seconds'
                   %(epoch, avg_loss/number_of_batches, (epoch_time_end-epoch_time_start)))